# 1. Загрузка и ознакомление с данными

In [23]:
import numpy as np 
import pandas as pd 
import datetime


In [24]:
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [25]:




from  phik import significance_matrix #mutual_info_regression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

In [26]:
import warnings
warnings.filterwarnings('ignore')

<h3>Загрузим таблицы</h3>

In [27]:
autodate = pd.read_csv('/kaggle/input/cars-price-prediction/train.csv')
autodate_test = pd.read_csv('/kaggle/input/cars-price-prediction/test.csv')

In [28]:
sample_submission = pd.read_csv('/kaggle/input/cars-price-prediction/sample_submission.csv')

<h3>Проверим соответствие данных в датасетах</h3>

In [29]:
print(autodate.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB
None


In [30]:
print(autodate_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110058 entries, 0 to 110057
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  int64  
 1   make          107997 non-null  object 
 2   model         107979 non-null  object 
 3   trim          107944 non-null  object 
 4   body          107464 non-null  object 
 5   transmission  97047 non-null   object 
 6   vin           110058 non-null  object 
 7   state         110058 non-null  object 
 8   condition     107679 non-null  float64
 9   odometer      110039 non-null  float64
 10  color         109900 non-null  object 
 11  interior      109900 non-null  object 
 12  seller        110058 non-null  object 
 13  saledate      110058 non-null  object 
dtypes: float64(2), int64(1), object(11)
memory usage: 11.8+ MB
None


<h4>Вывод: Как и должно быть, типы соответствуют.</h4>

<h4>Ознакомимся со сводкой по столбцам</h4>

In [31]:
#autodate.describe(include = "all")

# 2. Предварительная обработка

<h3>Упростим структуру даты, убрав данные часового пояса</h3>

In [32]:
autodate['saledate'] = pd.to_datetime(autodate['saledate'], utc=True).dt.date
autodate_test['saledate'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.date

In [33]:
autodate.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4.2,111041.0,black,black,santander consumer,12500,2015-06-01
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,3.5,31034.0,black,black,ars/avis budget group,14500,2015-02-24
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,2015-06-09
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,2015-06-16
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2.0,127709.0,black,black,purple heart,1300,2015-02-02


In [34]:
autodate_test.head()

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,saledate
0,2005,Cadillac,CTS,Base,Sedan,automatic,1g6dp567450124779,ca,2.7,116970.0,silver,black,lexus of stevens creek,2015-01-13
1,2014,GMC,Savana Cargo,2500,Van,NaN,1gtw7fca7e1902207,pa,4.4,6286.0,white,gray,u-haul,2015-02-26
2,2013,Nissan,Murano,S,SUV,automatic,jn8az1mw6dw303497,oh,4.6,11831.0,gray,black,nissan-infiniti lt,2015-02-23
3,2013,Chevrolet,Impala,LS Fleet,Sedan,automatic,2g1wf5e34d1160703,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,2015-03-05
4,2013,Nissan,Titan,SV,Crew Cab,automatic,1n6aa0ec3dn301209,tn,2.9,31083.0,black,black,nissan north america inc.,2015-06-02


<h3>Проверим наличие дупликатов и пропусков в данных и сделаем предположение о способах их устранения</h3>

In [35]:
autodate.shape

(440236, 15)

In [36]:
autodate_test.shape

(110058, 14)

<h4>Удаляем колонку  "vin". Она не нужна для расчетов</h4>

In [37]:
autodate.drop(columns='vin', inplace=True)
autodate_test.drop(columns='vin', inplace=True)

<h4>Проверим уникальные значения на скрытые дупликаты</h4>

In [38]:
print(autodate['make'].unique())

['Ford' 'Nissan' 'HUMMER' 'Lincoln' 'pontiac' 'Hyundai' nan 'Buick'
 'Chevrolet' 'Honda' 'Acura' 'Cadillac' 'GMC' 'Saab' 'Dodge'
 'Mercedes-Benz' 'Toyota' 'Volkswagen' 'BMW' 'Infiniti' 'Chrysler' 'Kia'
 'Jaguar' 'Subaru' 'Jeep' 'Lexus' 'mercedes' 'Scion' 'FIAT' 'Suzuki'
 'Mazda' 'Volvo' 'Audi' 'MINI' 'Isuzu' 'Mitsubishi' 'smart' 'Pontiac'
 'Porsche' 'subaru' 'ford' 'Land Rover' 'chrysler' 'Saturn' 'mazda'
 'dodge' 'Ram' 'Oldsmobile' 'hyundai' 'Mercury' 'Bentley' 'toyota'
 'lincoln' 'Fisker' 'nissan' 'chevrolet' 'honda' 'porsche' 'mitsubishi'
 'lexus' 'bmw' 'Maserati' 'acura' 'jeep' 'mercury' 'Tesla' 'landrover'
 'vw' 'cadillac' 'buick' 'gmc truck' 'land rover' 'volkswagen'
 'Rolls-Royce' 'audi' 'Ferrari' 'suzuki' 'Plymouth' 'oldsmobile'
 'Lamborghini' 'gmc' 'Geo' 'ford truck' 'Aston Martin' 'plymouth' 'Daewoo'
 'maserati' 'kia' 'mercedes-b' 'dodge tk' 'mazda tk' 'dot' 'airstream']


In [39]:
print(autodate_test['make'].unique())

['Cadillac' 'GMC' 'Nissan' 'Chevrolet' 'Volkswagen' 'Hyundai' 'Ford'
 'Infiniti' 'Subaru' 'Kia' 'BMW' nan 'Mitsubishi' 'Dodge' 'Oldsmobile'
 'Volvo' 'Pontiac' 'Acura' 'Honda' 'Toyota' 'Audi' 'Jeep' 'Lincoln'
 'Chrysler' 'Mercedes-Benz' 'Ram' 'Scion' 'Mercury' 'Lexus' 'Saturn'
 'Mazda' 'Porsche' 'MINI' 'smart' 'Buick' 'HUMMER' 'ford' 'chrysler'
 'Jaguar' 'mitsubishi' 'FIAT' 'Saab' 'mazda' 'Suzuki' 'Land Rover' 'bmw'
 'cadillac' 'oldsmobile' 'honda' 'acura' 'buick' 'landrover' 'Geo' 'lexus'
 'dodge' 'chevrolet' 'chev truck' 'nissan' 'Bentley' 'jeep' 'mercury'
 'toyota' 'gmc' 'Isuzu' 'Aston Martin' 'Maserati' 'land rover' 'kia'
 'subaru' 'Daewoo' 'vw' 'volkswagen' 'Tesla' 'mercedes' 'Ferrari'
 'hyundai' 'pontiac' 'Fisker' 'porsche' 'Rolls-Royce' 'Lotus' 'Plymouth'
 'hyundai tk' 'lincoln' 'ford tk' 'audi']


<h4>Прослеживаются дупликаты. Напишем функцию для их устранения</h4>

In [40]:
def replace_wrong_values(wrong_values, correct_value): 
    for wrong_value in wrong_values: 
        autodate['make'] = autodate['make'].replace(wrong_value, correct_value)
        autodate_test['make'] = autodate_test['make'].replace(wrong_value, correct_value)

In [41]:
#def replace_wrong_values1(wrong_values, correct_value): 
 #   for wrong_value in wrong_values: 
  #      autodate_test['make'] = autodate_test['make'].replace(wrong_value, correct_value)

<h4> Сформируем списки дупликатов и основных значений</h4>

In [42]:
duplicates26 = ['mercury']   
name26 = 'Mercury' 
replace_wrong_values(duplicates26,name26) 
duplicates25 = ['lincoln']   
name25 = 'Lincoln' 
replace_wrong_values(duplicates25,name25) 
duplicates24 = ['lexus']     
name24 = 'Lexus'   
replace_wrong_values(duplicates24,name24) 
duplicates23 = ['maserati']  
name23 = 'Maserati'  
replace_wrong_values(duplicates23,name23) 
duplicates22 = ['plymouth']  
name22 = 'Plymouth'  
replace_wrong_values(duplicates22,name22) 
        
duplicates21 = ['mazda', 'mazda tk']  
name21 = 'Mazda'  
replace_wrong_values(duplicates21,name21)
duplicates20 = ['audi'] 
name20 = 'Audi'         
replace_wrong_values(duplicates20,name20)
duplicates19 = ['porsche'] 
name19 = 'Porsche'  
replace_wrong_values(duplicates19,name19)
duplicates18 = ['mitsubishi'] 
name18 = 'Mitsubishi' 
replace_wrong_values(duplicates18,name18)
duplicates17 = ['landrover', 'land rover'] 
name17 = 'Land Rover'  
replace_wrong_values(duplicates17,name17)

duplicates16 = ['mercedes-b', 'mercedes'] 
name16 = 'Mercedes-Benz'  
replace_wrong_values(duplicates16,name16)
duplicates15 = ['volkswagen', 'vw']  
name15 = 'Volkswagen'  
replace_wrong_values(duplicates15,name15)
duplicates14 = ['bmw']  
name14 = 'BMW'  
replace_wrong_values(duplicates14,name14)
duplicates13 = ['kia']  
name13 = 'Kia'  
replace_wrong_values(duplicates13,name13)
duplicates12 = ['suzuki']  
name12 = 'Suzuki'  
replace_wrong_values(duplicates12,name12)

duplicates11 = ['honda']  
name11 = 'Honda'  
replace_wrong_values(duplicates11,name11)
duplicates10 = ['acura']  
name10 = 'Acura'  
replace_wrong_values(duplicates10,name10)
duplicates9 = ['cadillac'] 
name9 = 'Cadillac'  
replace_wrong_values(duplicates9,name9)
duplicates8 = ['gmc truck', 'gmc']  
name8 = 'GMC'  
replace_wrong_values(duplicates8,name8)
duplicates7 = ['dodge', 'dodge tk']  
name7 = 'Dodge'  
replace_wrong_values(duplicates7,name7)

duplicates1 = ['ford', 'ford truck', 'ford tk']  
name1 = 'Ford'  
replace_wrong_values(duplicates1,name1)
duplicates2 = ['nissan']   
name2 = 'Nissan'  
replace_wrong_values(duplicates2,name2)
duplicates3 = ['pontiac']  
name3 = 'Pontiac' 
replace_wrong_values(duplicates3,name3)
duplicates4 = ['hyundai']   
name4 = 'Hyundai'  
replace_wrong_values(duplicates4,name4)
duplicates5 = ['buick']  
name5 = 'Buick'
replace_wrong_values(duplicates5,name5)
duplicates6 = ['chevrolet','chev truck'] 
name6 = 'Chevrolet'  
replace_wrong_values(duplicates6,name6) 

duplicates27 = ['dodge tk', 'dodge'] 
name27 = 'Dodge'  
replace_wrong_values(duplicates27,name27)
duplicates28 = ['hyundai tk'] 
name28 = 'Hyundai'  
replace_wrong_values(duplicates28,name28)        

In [43]:
print(autodate['make'].unique())

['Ford' 'Nissan' 'HUMMER' 'Lincoln' 'Pontiac' 'Hyundai' nan 'Buick'
 'Chevrolet' 'Honda' 'Acura' 'Cadillac' 'GMC' 'Saab' 'Dodge'
 'Mercedes-Benz' 'Toyota' 'Volkswagen' 'BMW' 'Infiniti' 'Chrysler' 'Kia'
 'Jaguar' 'Subaru' 'Jeep' 'Lexus' 'Scion' 'FIAT' 'Suzuki' 'Mazda' 'Volvo'
 'Audi' 'MINI' 'Isuzu' 'Mitsubishi' 'smart' 'Porsche' 'subaru'
 'Land Rover' 'chrysler' 'Saturn' 'Ram' 'Oldsmobile' 'Mercury' 'Bentley'
 'toyota' 'Fisker' 'Maserati' 'jeep' 'Tesla' 'Rolls-Royce' 'Ferrari'
 'Plymouth' 'oldsmobile' 'Lamborghini' 'Geo' 'Aston Martin' 'Daewoo' 'dot'
 'airstream']


In [44]:
print(autodate_test['make'].unique())

['Cadillac' 'GMC' 'Nissan' 'Chevrolet' 'Volkswagen' 'Hyundai' 'Ford'
 'Infiniti' 'Subaru' 'Kia' 'BMW' nan 'Mitsubishi' 'Dodge' 'Oldsmobile'
 'Volvo' 'Pontiac' 'Acura' 'Honda' 'Toyota' 'Audi' 'Jeep' 'Lincoln'
 'Chrysler' 'Mercedes-Benz' 'Ram' 'Scion' 'Mercury' 'Lexus' 'Saturn'
 'Mazda' 'Porsche' 'MINI' 'smart' 'Buick' 'HUMMER' 'chrysler' 'Jaguar'
 'FIAT' 'Saab' 'Suzuki' 'Land Rover' 'oldsmobile' 'Geo' 'Bentley' 'jeep'
 'toyota' 'Isuzu' 'Aston Martin' 'Maserati' 'subaru' 'Daewoo' 'Tesla'
 'Ferrari' 'Fisker' 'Rolls-Royce' 'Lotus' 'Plymouth']


<h4>Проверим пропуски в данных<h4>

In [45]:
print(f'''Пропуски в данных:
{autodate.isna().sum()}''')

Пропуски в данных:
year                0
make             8043
model            8123
trim             8337
body            10393
transmission    51461
state               0
condition        9405
odometer           69
color             586
interior          586
seller              0
sellingprice        0
saledate            0
dtype: int64


In [46]:
print(f'''Пропуски в данных:
{autodate_test.isna().sum()}''')

Пропуски в данных:
year                0
make             2061
model            2079
trim             2114
body             2594
transmission    13011
state               0
condition        2379
odometer           19
color             158
interior          158
seller              0
saledate            0
dtype: int64


<h4>В обоих выборках похожие пропуски, поэтому методы устранения будут одинаковы.</h4>

<h3>1. Имеем пропуски в таких важных столбцах, как make, model, trim, body. Общий объем пропусков равен примерно 2%.</h3>
<h4>Решение: Заполним произвольными данными.</h4>

In [47]:
autodate = autodate.fillna({"make":"GT3", "model":"T", "trim":"fr", "body": "tipee"})
autodate_test = autodate_test.fillna({"make":"GT3", "model":"T", "trim":"fr", "body": "tipee"})

<h3>2. Около 0.0001 отсутствующих значений в колонках с цветом color, interior.</h3>
<h4>Решение: Заполним одним из топовых значений этих полей.</h4>

In [48]:
autodate['color'] = autodate['color'].replace('—', 'white').fillna(value='white')
autodate_test['color'] = autodate_test['color'].replace('—', 'white').fillna(value='white')

In [49]:
autodate['interior'] = autodate['interior'].replace('—', 'white').fillna(value='white')
autodate_test['interior'] = autodate_test['interior'].replace('—', 'white').fillna(value='white')

<h3>3. По статистике в США более 95% авто снабжены АКПП.</h3>
<h4>Решение: Заполним соответствующим значением колонку transmission.</h4>

In [50]:
autodate['transmission'] = autodate['transmission'].fillna(value='automatic')
autodate_test['transmission'] = autodate_test['transmission'].fillna(value='automatic')

<h3>4. Поле condition имеет около 2% отсутствующих значений</h3>
<h4>Решение: Заполним медианным значением.</h4>

In [51]:
autodate['condition'] = autodate['condition'].fillna(autodate['condition'].mean())
autodate_test['condition'] = autodate_test['condition'].fillna(autodate['condition'].mean())

<h3>5. Поле odometer вообще имеет около 0.00015% пропусков</h3>
<h4>Решение: Заполним средним значением.</h4>

In [52]:
autodate['odometer'] = autodate['odometer'].fillna(autodate['odometer'].mean())
autodate_test['odometer'] = autodate_test['odometer'].fillna(autodate_test['odometer'].mean())

In [53]:
#autodate_test['odometer'] = autodate_test['odometer'].astype('float')

In [54]:
autodate_test['odometer'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 110058 entries, 0 to 110057
Series name: odometer
Non-Null Count   Dtype  
--------------   -----  
110058 non-null  float64
dtypes: float64(1)
memory usage: 860.0 KB


In [55]:
autodate.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  object 
 2   model         440236 non-null  object 
 3   trim          440236 non-null  object 
 4   body          440236 non-null  object 
 5   transmission  440236 non-null  object 
 6   state         440236 non-null  object 
 7   condition     440236 non-null  float64
 8   odometer      440236 non-null  float64
 9   color         440236 non-null  object 
 10  interior      440236 non-null  object 
 11  seller        440236 non-null  object 
 12  sellingprice  440236 non-null  int64  
 13  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(10)
memory usage: 47.0+ MB


<h3>Проверим</h3>

In [56]:
print(f'''Пропуски в данных x:
{autodate.isna().sum()}''')

Пропуски в данных x:
year            0
make            0
model           0
trim            0
body            0
transmission    0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
sellingprice    0
saledate        0
dtype: int64


In [57]:
print(f'''Пропуски в данных x:
{autodate_test.isna().sum()}''')

Пропуски в данных x:
year            0
make            0
model           0
trim            0
body            0
transmission    0
state           0
condition       0
odometer        0
color           0
interior        0
seller          0
saledate        0
dtype: int64


In [58]:
autodate.head()

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,md,4.2,111041.0,black,black,santander consumer,12500,2015-06-01
1,2014,Ford,Fusion,SE,Sedan,automatic,mo,3.5,31034.0,black,black,ars/avis budget group,14500,2015-02-24
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,nj,2.2,35619.0,black,black,nissan-infiniti lt,9100,2015-06-09
3,2003,HUMMER,H2,Base,suv,automatic,tx,2.8,131301.0,gold,beige,wichita falls ford lin inc,13300,2015-06-16
4,2007,Ford,Fusion,SEL,Sedan,automatic,md,2.0,127709.0,black,black,purple heart,1300,2015-02-02


In [59]:
autodate_test.head()

,year,make,model,trim,body,transmission,state,condition,odometer,color,interior,seller,saledate
0,2005,Cadillac,CTS,Base,Sedan,automatic,ca,2.7,116970.0,silver,black,lexus of stevens creek,2015-01-13
1,2014,GMC,Savana Cargo,2500,Van,automatic,pa,4.4,6286.0,white,gray,u-haul,2015-02-26
2,2013,Nissan,Murano,S,SUV,automatic,oh,4.6,11831.0,gray,black,nissan-infiniti lt,2015-02-23
3,2013,Chevrolet,Impala,LS Fleet,Sedan,automatic,fl,2.3,57105.0,silver,black,onemain rem/auto club of miami inc dba north dad,2015-03-05
4,2013,Nissan,Titan,SV,Crew Cab,automatic,tn,2.9,31083.0,black,black,nissan north america inc.,2015-06-02


streamlit   (ede pycharm, visual studio cod)

<h4>Разложим дату на дополнительные фитчи</h4>

In [60]:
# Для трейна
autodate['sl_year'] = pd.to_datetime(autodate['saledate'], utc=True).dt.year
autodate['sl_quarter'] = pd.to_datetime(autodate['saledate'], utc=True).dt.quarter
autodate['s_month'] = pd.to_datetime(autodate['saledate'], utc=True).dt.month
autodate['sl_weekday'] = pd.to_datetime(autodate['saledate'], utc=True).dt.weekday
autodate['s_day'] = pd.to_datetime(autodate['saledate'], utc=True).dt.day

In [61]:
#Для теста
autodate_test['sl_year'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.year
autodate_test['sl_quarter'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.quarter
autodate_test['s_month'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.month
autodate_test['sl_weekday'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.weekday
autodate_test['s_day'] = pd.to_datetime(autodate_test['saledate'], utc=True).dt.day

In [62]:
date_decomp = pd.DataFrame({'s_day': autodate['s_day'],'sl_weekday': autodate['sl_weekday'],'s_month': autodate['s_month'], 'sl_quarter': autodate['sl_quarter'],'sl_year':autodate['sl_year'] })
date_decomp.head()

,s_day,sl_weekday,s_month,sl_quarter,sl_year
0,1,0,6,2,2015
1,24,1,2,1,2015
2,9,1,6,2,2015
3,16,1,6,2,2015
4,2,0,2,1,2015


In [63]:
date_decomp_test = pd.DataFrame({'s_day': autodate_test['s_day'],'sl_weekday': autodate_test['sl_weekday'],'s_month': autodate_test['s_month'], 'sl_quarter': autodate_test['sl_quarter'],'sl_year':autodate_test['sl_year'] })
date_decomp_test.head()

,s_day,sl_weekday,s_month,sl_quarter,sl_year
0,13,1,1,1,2015
1,26,3,2,1,2015
2,23,0,2,1,2015
3,5,3,3,1,2015
4,2,1,6,2,2015


In [64]:
autodate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  object 
 2   model         440236 non-null  object 
 3   trim          440236 non-null  object 
 4   body          440236 non-null  object 
 5   transmission  440236 non-null  object 
 6   state         440236 non-null  object 
 7   condition     440236 non-null  float64
 8   odometer      440236 non-null  float64
 9   color         440236 non-null  object 
 10  interior      440236 non-null  object 
 11  seller        440236 non-null  object 
 12  sellingprice  440236 non-null  int64  
 13  saledate      440236 non-null  object 
 14  sl_year       440236 non-null  int64  
 15  sl_quarter    440236 non-null  int64  
 16  s_month       440236 non-null  int64  
 17  sl_weekday    440236 non-null  int64  
 18  s_da

In [65]:
#Сбросим первоначальный стоблец с датой
autodate.drop(columns='saledate', inplace=True)
autodate_test.drop(columns='saledate', inplace=True)

In [66]:
# Разделим категориальные и количественные признаки в два разных датафрейма для теста
autodate_cat = autodate[['make', 'model', 'trim', 'body', 'transmission', 'state','color', 'interior', 'seller']] 
autodate_col = autodate[['year', 'condition', 'odometer', 'sellingprice']] 

In [67]:
print(autodate.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  object 
 2   model         440236 non-null  object 
 3   trim          440236 non-null  object 
 4   body          440236 non-null  object 
 5   transmission  440236 non-null  object 
 6   state         440236 non-null  object 
 7   condition     440236 non-null  float64
 8   odometer      440236 non-null  float64
 9   color         440236 non-null  object 
 10  interior      440236 non-null  object 
 11  seller        440236 non-null  object 
 12  sellingprice  440236 non-null  int64  
 13  sl_year       440236 non-null  int64  
 14  sl_quarter    440236 non-null  int64  
 15  s_month       440236 non-null  int64  
 16  sl_weekday    440236 non-null  int64  
 17  s_day         440236 non-null  int64  
dtypes: f

In [68]:
# для трейна
autodate_cat_test = autodate_test[['make', 'model', 'trim', 'body', 'transmission', 'state','color', 'interior', 'seller']] 
autodate_col_test = autodate_test[['year', 'condition', 'odometer']] 

In [69]:
#Преобразуем категориальные признаки и объеденим их в датафрем, создадим второй датафрейм для числовых признаков
encoder = OrdinalEncoder()
autodate_cat = pd.DataFrame(encoder.fit_transform(autodate_cat),columns=autodate_cat.columns) 
 #Для теста
autodate_cat_test = pd.DataFrame(encoder.fit_transform(autodate_cat_test),columns=autodate_cat_test.columns) 


In [70]:
autodate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  object 
 2   model         440236 non-null  object 
 3   trim          440236 non-null  object 
 4   body          440236 non-null  object 
 5   transmission  440236 non-null  object 
 6   state         440236 non-null  object 
 7   condition     440236 non-null  float64
 8   odometer      440236 non-null  float64
 9   color         440236 non-null  object 
 10  interior      440236 non-null  object 
 11  seller        440236 non-null  object 
 12  sellingprice  440236 non-null  int64  
 13  sl_year       440236 non-null  int64  
 14  sl_quarter    440236 non-null  int64  
 15  s_month       440236 non-null  int64  
 16  sl_weekday    440236 non-null  int64  
 17  s_day         440236 non-null  int64  
dtypes: f

In [71]:
autodate_col = pd.DataFrame((autodate_col),columns=autodate_col.columns)
autodate_col_test = pd.DataFrame((autodate_col_test),columns=autodate_col_test.columns) 

In [72]:
#Объеденим обе части таблицы
autodate = pd.concat([autodate_cat, autodate_col,date_decomp], axis = 1)
autodate = autodate[['year','make','model','trim', 'body','transmission','state', 'color', 'condition', 'odometer', 'interior', 'seller', 'sellingprice','s_day','sl_weekday','s_month','sl_quarter', 'sl_year']]
autodate.head()

,year,make,model,trim,body,transmission,state,color,condition,odometer,interior,seller,sellingprice,s_day,sl_weekday,s_month,sl_quarter,sl_year
0,2011,14.0,229.0,1229.0,79.0,0.0,12.0,1.0,4.2,111041.0,1.0,10697.0,12500,1,0,6,2,2015
1,2014,14.0,297.0,1202.0,36.0,0.0,15.0,1.0,3.5,31034.0,1.0,736.0,14500,24,1,2,1,2015
2,2012,36.0,617.0,69.0,76.0,0.0,19.0,1.0,2.2,35619.0,1.0,8503.0,9100,9,1,6,2,2015
3,2003,18.0,347.0,553.0,79.0,0.0,33.0,6.0,2.8,131301.0,0.0,12845.0,13300,16,1,6,2,2015
4,2007,14.0,297.0,1229.0,36.0,0.0,12.0,1.0,2.0,127709.0,1.0,9750.0,1300,2,0,2,1,2015


In [73]:
#Объеденим обе части для теста
autodate_test = pd.concat([autodate_cat_test, autodate_col_test, date_decomp], axis = 1)
autodate_test = autodate_test[['year','make','model','trim', 'body','transmission','state', 'color','condition', 'odometer', 'interior', 'seller','s_day','sl_weekday','s_month','sl_quarter', 'sl_year']]
autodate_test.head()

,year,make,model,trim,body,transmission,state,color,condition,odometer,interior,seller,s_day,sl_weekday,s_month,sl_quarter,sl_year
0,2005.0,6.0,114.0,400.0,33.0,0.0,3.0,15.0,2.7,116970.0,1.0,4128.0,1,0,6,2,2015
1,2014.0,15.0,557.0,143.0,38.0,0.0,28.0,17.0,4.4,6286.0,6.0,7163.0,24,1,2,1,2015
2,2013.0,36.0,425.0,938.0,32.0,0.0,24.0,7.0,4.6,11831.0,1.0,4992.0,9,1,6,2,2015
3,2013.0,7.0,327.0,759.0,33.0,0.0,5.0,15.0,2.3,57105.0,1.0,5120.0,16,1,6,2,2015
4,2013.0,36.0,619.0,1070.0,9.0,0.0,32.0,1.0,2.9,31083.0,1.0,4973.0,2,0,2,1,2015


In [74]:
autodate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          440236 non-null  float64
 2   model         440236 non-null  float64
 3   trim          440236 non-null  float64
 4   body          440236 non-null  float64
 5   transmission  440236 non-null  float64
 6   state         440236 non-null  float64
 7   color         440236 non-null  float64
 8   condition     440236 non-null  float64
 9   odometer      440236 non-null  float64
 10  interior      440236 non-null  float64
 11  seller        440236 non-null  float64
 12  sellingprice  440236 non-null  int64  
 13  s_day         440236 non-null  int64  
 14  sl_weekday    440236 non-null  int64  
 15  s_month       440236 non-null  int64  
 16  sl_quarter    440236 non-null  int64  
 17  sl_year       440236 non-null  int64  
dtypes: f

In [75]:
autodate_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          110058 non-null  float64
 1   make          110058 non-null  float64
 2   model         110058 non-null  float64
 3   trim          110058 non-null  float64
 4   body          110058 non-null  float64
 5   transmission  110058 non-null  float64
 6   state         110058 non-null  float64
 7   color         110058 non-null  float64
 8   condition     110058 non-null  float64
 9   odometer      110058 non-null  float64
 10  interior      110058 non-null  float64
 11  seller        110058 non-null  float64
 12  s_day         440236 non-null  int64  
 13  sl_weekday    440236 non-null  int64  
 14  s_month       440236 non-null  int64  
 15  sl_quarter    440236 non-null  int64  
 16  sl_year       440236 non-null  int64  
dtypes: float64(12), int64(5)
memory usage: 57.1 MB


# 3. Разработка синтетических данных

# 4. Выбор и обучение моделей

<h4>Разделим данные на выборки</h4>

In [78]:
#Выделим таргет и фьючи:
features = autodate.drop('sellingprice', axis=1)
target = autodate.sellingprice

In [79]:
#разделим данные на выборки:
features_train, features_test, target_train, target_test = train_test_split(features,
                                                                            target, 
                                                                            test_size=.25,
                                                                            random_state=12345)

<h4>Выберем Модель CatBoostRegressor</h4>

In [81]:
%%time

model_cbr = CatBoostRegressor() 
parameters = [{'learning_rate':[.1, .5, .8], 'random_state':[12345], 'verbose':[False]}]

gscv = GridSearchCV(model_cbr, parameters, scoring='neg_mean_squared_error')
gscv.fit(features_train, target_train)

print(gscv.best_params_)

mts = gscv.cv_results_['mean_test_score']
    
gscv_rsme_cbr = (max(mts) * -1) ** .5
gscv_rsme_cbr

{'learning_rate': 0.5, 'random_state': 12345, 'verbose': False}
CPU times: user 24min 4s, sys: 14.4 s, total: 24min 18s
Wall time: 12min 42s


2088.073852125626

In [82]:
%%time
model_cbr = CatBoostRegressor(learning_rate=.5, random_state=12345, verbose=False) #подставил лучшие параметры
model_cbr.fit(features_train, target_train)

CPU times: user 1min 50s, sys: 972 ms, total: 1min 51s
Wall time: 58.2 s


In [87]:
%%time

target_predict = model_cbr.predict(features_test)

CPU times: user 222 ms, sys: 7.99 ms, total: 230 ms
Wall time: 126 ms


In [88]:
out_test = autodate_test

In [89]:
%%time
main_predict = model_cbr.predict(out_test)

CPU times: user 891 ms, sys: 4.99 ms, total: 896 ms
Wall time: 470 ms


In [90]:
 #Определяем функцию MAPE, из sklearn ее удалили
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true) * 100)

In [99]:
#submission = pd.read_csv('/kaggle/input/cars-price-prediction/submission.csv')
#submission['vin'] = autodate['vin']
submission = pd.DataFrame()
#submission['vin'] = autodate_test['vin']
submission['sellingprice'] = model_cbr.predict(features_test)
submission.to_csv(r'/kaggle/input/cars-price-prediction/sample_submission.csv',index = False)

OSError: [Errno 30] Read-only file system: '/kaggle/input/cars-price-prediction/sample_submission.csv'

In [350]:

submission['price'] = np.exp(model_cbr.predict(features_test))*0.96

EmptyDataError: No columns to parse from file

In [ ]:
submission.to_csv(r'/kaggle/input/cars-price-prediction/submission.csv',index = False)

In [ ]:
# Проверяем Catboost на валидационной выборке
MAPE = mape(np.exp(y_val), np.exp(cbr.predict(X_val)))
print(f'Mean Absolute Percentage Error: {MAPE}')

# Predictions on test sample
submission['price'] = np.exp(cbr.predict(X_test))*0.96
submission.to_csv('/kaggle/input/cars-price-prediction/submission.csv', index=False)

In [ ]:
rmse_cbr = mean_squared_error(target_test, target_predict) ** .5
rmse_cbr

# 5.Выбор лучшей модели

# 6.Вывод

<h4>Закодируем категориальные переменные используя OneHotEncoder</h4>

In [ ]:
features_train = train_data.drop(['sellingprice'], axis=1)
target_train = train_data['sellingprice']
features_valid = train_data.drop(['sellingprice'], axis=1)
target_valid = train_data['sellingprice']